In [1]:
import numpy as np
import pandas as pd
import math
#import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
#from sklearn.metrics import confusion_matrix
from scipy.stats import *

#%matplotlib inline

In [2]:
nombres = ['make','adress','all','3d','our','over','remove','internet','order','mail',
           'receive','will','people','report','addresses','free','busisnes','email','you',
           'credit','your','font','000','money','hp','hpl','george','650','lab','labs','telnet',
           '857','data','415','85','technology','1999','parts','pm','direct','cs','meeting',
           'original','project','re','edu','table','conference',';','(','[','!','$','#',
           'capital_run_length_average','capital_run_length_longest','capital_run_length_total','spam'
]

df = pd.read_csv("datos.csv", names = nombres) # El csv no tenía nombres puestos

# Hacemos los cálculos del entrenamiento

In [3]:
entrenamiento, prueba = train_test_split(df, test_size=0.2)#, random_state=18)

media_no_spam = entrenamiento[entrenamiento.spam==0].mean()
media_spam = entrenamiento[entrenamiento.spam==1].mean()

std_no_spam = entrenamiento[entrenamiento.spam==0].std()
std_spam = entrenamiento[entrenamiento.spam==1].std()

x_entrenamiento = entrenamiento.iloc[:,0:entrenamiento.shape[1]-1]
y_entrenamiento = entrenamiento.spam

x_prueba = prueba.iloc[:,0:prueba.shape[1]-1]
y_prueba = prueba.spam

pro_den_fun = norm(media_no_spam[0],std_no_spam[0])

# Definimos las funciones

In [4]:
def sets(data_frame):
    """
    Esta función divide los data frames en entrenamiento y prueba
    """
    entrenamiento, prueba = train_test_split(data_frame, test_size=0.2)#, random_state=18)
    return entrenamiento, prueba


def calcularProbabilidad(x, mean, std):
    """
    Esta función calcula la probabilidad de que la variable 'x' pertenezca a una pdf normal con media = mean y
    desviación estándar = std
    """
    exponente = math.exp(-(math.pow(x-mean,2)/(2*math.pow(std,2))))
    return (1 / (math.sqrt(2*math.pi) * std)) * exponente


def probabilidad_pertenencia_spam(n,df=x_entrenamiento):
    """
    Esta función sirve para obtener la probabilidad de que una observación pertenezca a la categoría de SPAM
    
    Función de: 
    n := Número de la observación
    df := El valor default es x_entrenamiento. Para hacer la inferencia hay que usar x_prueba
    """
    hola = np.zeros(df.shape[1])
    for i in range(0,len(hola)):
        if df.iloc[n,i] != 0:
            x = df.iloc[n,i]
            mean = media_spam[i]
            std = std_spam[i] 
            si = calcularProbabilidad(x, mean, std)
            hola[i] = si
    return(np.sum(hola)/len(hola))

def probabilidad_pertenencia_no_spam(n,df=x_entrenamiento):
    """
    Esta función sirve para obtener la probabilidad de que una observación no pertenezca a la categoría de SPAM
    
    Función de: 
    n := Número de la observación
    df := El valor default es x_entrenamiento. Para hacer la inferencia hay que usar x_prueba
    """
    hola = np.zeros(df.shape[1])
    for i in range(0,len(hola)):
        if df.iloc[n,i] != 0:
            x = df.iloc[n,i]
            mean = media_no_spam[i]
            std = std_no_spam[i] 
            si = calcularProbabilidad(x, mean, std)
            hola[i] = si
    return(np.sum(hola)/len(hola))

def es_o_no_es(df):
    es_o_no_es_prueba = np.zeros(df.shape[0])
    for i in range(0,df.shape[0]):
        if probabilidad_pertenencia_spam(i,df) > probabilidad_pertenencia_no_spam(i,df):
            es_o_no_es_prueba[i] = 1
    return es_o_no_es_prueba

def precision(df):
    es = es_o_no_es(df)
    probabilidad = (1- sum(abs(y_prueba - es))/len(es))*100
    print('Pude predecir el spam el {0}% de las veces').format(probabilidad)

# Hacemos la prueba

In [5]:
precision(x_prueba)

Pude predecir el spam el 70.901194354% de las veces
